In [1]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def write_contrast(src, out_file):
    cooc_mat = np.zeros((256, 256))

    im = Image.open(src)
    x, y = im.size
    im = im.load()
    
    for j in range(1, y, 1):
        for i in range(x):
            north = im[i, j-1]
            south = im[i, j]
            cooc_mat[north, south] += 1
            
    contrast_val = 0;
    
    for i in range(256):
        for j in range(256):
            contrast_val += cooc_mat[i, j] * ((i - j)**2)
    f = open(out_file, "a+")
    f.write("%s %d\n" % (src, contrast_val))
    f.close()

In [3]:
def contrast(src, A, B, C, D, out, cont_out):
    im = Image.open(src)
    x, y = im.size
    im = im.load()
    r1_slope = (B-0)/(A-0)
    r2_slope = (D-B)/(C-A)
    r3_slope = (255-D)/(255-C)
    out_img = Image.new('L', (x, y))
    out_im = []
    for i in range(y):
        for j in range(x):
            curr_pix = im[j, i]
            if curr_pix <= A:
                out_im.append(curr_pix * r1_slope)
            elif curr_pix <= C:
                out_im.append(curr_pix * r2_slope + (D-C*r2_slope))
            else:
                out_im.append(curr_pix * r3_slope + (255-255*r3_slope))
    out_img.putdata(out_im)
    out_img.save(out)
    write_contrast(out, cont_out)

In [4]:
contrast("Ocean.bmp", 30, 20,180, 230, "Ocean_a.bmp", "contrast.txt")
contrast("Ocean.bmp", 70, 20, 140, 240, "Ocean_b.bmp", "contrast.txt")

In [5]:
def integral_img(src, out, view):
    im = Image.open(src)
    x, y = im.size
    im = im.load()
    if view:
        out_img = Image.new('L', (x, y))
    else:
        out_img = Image.new('I', (x, y))
    out_im = []
    
    for i in range(y):
        for j in range(x):
            curr_pix = im[j, i]
            _sum = 0
            _sum += curr_pix
            if(i > 0):
                _sum += out_im[(i-1)*x + j]
            if (j > 0):
                _sum += out_im[i*x + j-1]
            if (i > 0 and j > 0):
                _sum -= out_im[(i-1)*x + j-1]
            out_im.append(_sum)
    if view:
        _max = max(out_im)
        for i in range(len(out_im)):
            out_im[i] = (out_im[i]/ _max)*255
        
    out_img.putdata(out_im)
    out_img.save(out)

In [6]:
integral_img("Cameraman_noise.bmp", "Camera_Integ.jpg", True)

In [7]:
def avg_integ(src, out, K_size):
    integral_img(src, "temp.tiff", False)
    int_img = Image.open("Camera_Integ.tiff")
    int_im = int_img.load()
    
    im = Image.open(src)
    x, y = im.size
    im = im.load()
    out_img = Image.new('L', (x-K_size + 1, y - K_size + 1))
    out_im = []
    for i in range((K_size-1)//2, y - (K_size-1)//2, 1):
        for j in range((K_size-1)//2, x - (K_size-1)//2, 1):
            i_c, j_c = (i + (K_size-1)//2, j + (K_size-1)//2)
            final = int_im[j_c, i_c]
            if (j > K_size//2):
                final -= int_im[j_c - K_size, i_c]
            if (i > K_size//2):
                final -= int_im[j_c, i_c - K_size]
            if (i > K_size//2 and j > K_size//2):
                final += int_im[j_c - K_size, i_c - K_size]
            out_im.append(final/(K_size**2))
    out_img.putdata(out_im)
    out_img.save(out)

In [8]:
avg_integ("Cameraman_noise.bmp", "Camera_Filt_3.jpg", 3)
avg_integ("Cameraman_noise.bmp", "Camera_Filt_5.jpg", 5)

Problem2, C.
    The advantage of using the integral image for averaging is that the cost of applying the kernel on the image will be constant not dependent on the kernel size. 
As the cost of calculating the sum of any area in the image will always cost the same as it only needs the 4 pixels surroinding the corners of the area in the integral image.